In [ ]:
import sys
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tf_pose import common
import tf_slim as slim
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

In [ ]:
def show_keypoints(image, hum, human=1, color='orange', showBG = True):
    if human == 0: human = 1
    num_hum = len(hum)
    keypoints = str(str(str(hum[human-1]).split('BodyPart:')[1:]).split('-')).split(' score=')
    keypoints_list=[]
    for i in range (len(keypoints)-1): 
        pnt = keypoints[i][-11:-1]
        pnt = tuple(map(float, pnt.split(', ')))
        keypoints_list.append(pnt)

    keypts_array = np.array(keypoints_list)
    keypts_array = keypts_array*(image.shape[1],image.shape[0])
    keypts_array = keypts_array.astype(int)
    keypts_array

    plt.figure(figsize=(10,10))
    plt.axis([0, image.shape[1], 0, image.shape[0]])  
    plt.scatter(*zip(*keypts_array), s=200, color=color, alpha=0.6)
    if showBG:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)      
    plt.imshow(image)
    ax=plt.gca() 
    ax.set_ylim(ax.get_ylim()[::-1]) 
    ax.xaxis.tick_top() 
    plt.title('Keypoints Person [{}] from {} humans detected\n'.format(human, num_hum))
    plt.grid();

    for i, txt in enumerate(keypts_array):
        ax.annotate(i, (keypts_array[i][0]-5, keypts_array[i][1]+5))
            
    return keypts_array

In [ ]:
model='mobilenet_v2_large'
resize='480x432'
w, h = model_wh(resize)

In [ ]:
e = TfPoseEstimator(get_graph_path(model), target_size=(w, h))

In [ ]:
video_path = '/Users/andreibirladeanu/Documents/Data/videos_trial/1043_meal.mp4'

In [ ]:
cap = cv2.VideoCapture(video_path)

In [ ]:
showBG = True
fps_time = 0
while True:
    ret_val, image = cap.read()
    humans = e.inference(image,
                         resize_to_default=(w > 0 and h > 0),
                         upsample_size=4.0)
    keypoints = str(str(str(humans[0]).split('BodyPart:')[1:]).split('-')).split(' score=')
    print(keypoints)
    keypoints_list=[]
    for i in range (len(keypoints)-1): 
        pnt = keypoints[i][-11:-1]
        pnt = tuple(map(float, pnt.split(', ')))
        print(pnt)
        keypoints_list.append(pnt)
    keypts_array = np.array(keypoints_list)
    print(keypts_array)
    keypts_array = keypts_array*(image.shape[1],image.shape[0])
    print(image.shape)
    keypts_array = keypts_array.astype(int)
    print(f"keypoints for person 1 {keypts_array}")
    minimum = np.amin(keypts_array, axis=0)
    maximum = np.amax(keypts_array, axis=0)
        
    cv2.rectangle(image,(minimum[0]-20, minimum[1]-20),(maximum[0]+20,maximum[1]+20),(0,255,0),2)
    
    #image = np.zeros(image.shape)
    image = TfPoseEstimator.draw_humans(image, humans, imgcopy=True)
   # cv2.putText(image, "FPS: %f" % (1.0 / (time.time() - fps_time)), (10, 10),
    #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.imshow('tf-pose-estimation result', image)
    #fps_time = time.time()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()